In [1]:
import sys
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost as xgb

from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [2]:
train_df.head()
train_df.info()
train_df.describe()
train_df.describe(include=['O'])
train_df.Dependents.unique()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
count,614,601,611,599,614,582,614,614
unique,614,2,2,4,2,2,3,2
top,LP001120,Male,Yes,0,Graduate,No,Semiurban,Y
freq,1,489,398,345,480,500,233,422


array(['0', '1', '2', '3+', nan], dtype=object)

In [3]:
train_df['Gender'].fillna(train_df['Gender'].mode()[0], inplace=True)
train_df['Married'].fillna(train_df['Married'].mode()[0], inplace=True)
train_df['Dependents'].fillna(train_df['Dependents'].mode()[0], inplace=True)
train_df['Self_Employed'].fillna(train_df['Self_Employed'].mode()[0], inplace=True)
train_df['LoanAmount'].fillna(train_df['LoanAmount'].mean(), inplace=True)
train_df['Loan_Amount_Term'].fillna(train_df['Loan_Amount_Term'].mean(), inplace=True)
train_df['Credit_History'].fillna(train_df['Credit_History'].mean(), inplace=True)

In [4]:
train_df.info()
y_train = train_df['Loan_Status']
y_train.head()
y_train = y_train.apply({'Y':1, 'N':0}.get)
x_train = train_df.drop(['Loan_Status'], axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null object
Married              614 non-null object
Dependents           614 non-null object
Education            614 non-null object
Self_Employed        614 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           614 non-null float64
Loan_Amount_Term     614 non-null float64
Credit_History       614 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


0    Y
1    N
2    Y
3    Y
4    Y
Name: Loan_Status, dtype: object

In [5]:
train_df1 = x_train.copy();
test_df1 = test_df.copy();

le = LabelEncoder()
train_df1['Loan_ID'] = le.fit_transform(train_df['Loan_ID'])
test_df1['Loan_ID'] = le.fit_transform(test_df['Loan_ID'])

train_df1 = pd.get_dummies(train_df1)
test_df1 = pd.get_dummies(test_df1)

train_df1.info()
train_df1.head()
x_train = train_df1
x_test = test_df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 21 columns):
Loan_ID                    614 non-null int64
ApplicantIncome            614 non-null int64
CoapplicantIncome          614 non-null float64
LoanAmount                 614 non-null float64
Loan_Amount_Term           614 non-null float64
Credit_History             614 non-null float64
Gender_Female              614 non-null uint8
Gender_Male                614 non-null uint8
Married_No                 614 non-null uint8
Married_Yes                614 non-null uint8
Dependents_0               614 non-null uint8
Dependents_1               614 non-null uint8
Dependents_2               614 non-null uint8
Dependents_3+              614 non-null uint8
Education_Graduate         614 non-null uint8
Education_Not Graduate     614 non-null uint8
Self_Employed_No           614 non-null uint8
Self_Employed_Yes          614 non-null uint8
Property_Area_Rural        614 non-null uint8
Property_Area

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0,5849,0.0,146.412162,360.0,1.0,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1
1,1,4583,1508.0,128.000000,360.0,1.0,0,1,0,1,...,1,0,0,1,0,1,0,1,0,0
2,2,3000,0.0,66.000000,360.0,1.0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
3,3,2583,2358.0,120.000000,360.0,1.0,0,1,0,1,...,0,0,0,0,1,1,0,0,0,1
4,4,6000,0.0,141.000000,360.0,1.0,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1


In [6]:
dtrain = xgb.DMatrix(x_train.values, label=y_train, missing=np.nan)
param = {'objective': 'binary:logistic', 'booster': 'gbtree', 'silent': 1,
		 'max_depth': 10, 'eta': 0.1,
		 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 20,
		 'max_delta_step': 0, 'gamma': 0}
num_round = 600

In [7]:
seeds = [1122, 2244, 3366, 4488, 5500]
test_preds = np.zeros((len(x_test), len(seeds)))

for run in range(len(seeds)):
	sys.stdout.write("\rXGB RUN:{}/{}".format(run+1, len(seeds)))
	sys.stdout.flush()
	param['seed'] = seeds[run]
	clf = xgb.train(param, dtrain, num_round)
	dtest = xgb.DMatrix(x_test.values, missing=np.nan)
	test_preds[:, run] = clf.predict(dtest)

test_preds = np.mean(test_preds, axis=1)

XGB RUN:5/5

In [8]:
test_df['Loan_Status'] = test_preds
test_df['Loan_Status'] = test_df['Loan_Status'] > 0.5
test_df['Loan_Status'] = test_df['Loan_Status'].apply({True:'Y', False:'N'}.get)
test_df.to_csv("../output/final_solution.csv", index=False)